In [1]:
import pandas as pd
from collections import Counter
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
pd.options.display.max_rows=None
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
file_path = "../data/en.openfoodfacts.org.products.csv.gz"

In [3]:
chunk_size = 10000

filtered_chunks = []

for chunk in pd.read_csv(
    file_path,
    chunksize=chunk_size,
    compression="gzip",
    sep="\t",
    engine="python",
    quoting=3,
):
    filtered_chunks_en = chunk[
        (chunk["countries_tags"] == "en:france") & (chunk["ingredients_tags"].notnull() & chunk["product_name"].notnull())
    ]
    filtered_chunks.append(filtered_chunks_en)

if filtered_chunks:
    data_en = pd.concat(filtered_chunks, axis=0)

In [4]:
# Find the different state of each rows
states_unique_values = set()
data_en["states_tags"].str.split(",").apply(states_unique_values.update)

states_df = pd.DataFrame(data=states_unique_values, columns=["states"])
display(states_df)

,states
0,en:quantity-completed
1,en:nutrition-facts-to-be-completed
2,en:expiration-date-to-be-completed
3,en:photos-to-be-validated
4,en:photos-uploaded
5,en:packaging-code-to-be-completed
6,en:ingredients-completed
7,en:packaging-to-be-completed
8,en:packaging-photo-to-be-selected
9,en:origins-to-be-completed


In [5]:
# Target sta
target_states = [
    "en:completed",
    "en:origins-completed",
    "en:ingredients-completed",
    "en:characteristics-completed",
]

# Construire une expression régulière pour les états
pattern = "|".join(target_states)

# Compter les lignes contenant au moins un des états cibles
count = data_en["states_tags"].str.contains(pattern).sum()

print("Nombre de lignes avec les états spécifiés:", count)

data_en_filtered = data_en[data_en["states_tags"].str.contains(pattern)]

Nombre de lignes avec les états spécifiés: 255537


In [6]:
len(data_en) - 258361

1181

In [7]:
describe_sample = data_en_filtered.describe(include="all").drop("top").T

In [8]:
describe_sample.T

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
count,255537.0,255537,255536,255537.0,255537,255537.0,255537,249517,255477.0,255477,255537,13865,72394,190388,130135,130135,130135,12100,241099,241092,220552,220552,220552,50434,50398,50397,61678,61655,159399,159399,159399,61551,61540,48041,0.0,51396,76756,112432,255537,255537,255537,255536,255537,255535,99877,0.0,59420,76867,75917,64571,62647.0,13222,255537.0,2,130552,130552,190433.0,255522,222182.0,255536,255537,198915,198915,198915,255537,255537,255537,3875,159029.0,255530,192600,188947.0,50071,8343,186866.0,183512,255537.0,249827.0,249827,220552,220552,237764,237764,222736,222736,181955,181955,84818.0,211997.0,229760.0,27.0,229518.0,227845.

In [9]:
threshold = 0.85
fill_threshold_percent = describe_sample[describe_sample["count"] > len(data_en_filtered) * threshold].T.columns
fill_threshold_percent

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime', 'product_name', 'brands',
       'brands_tags', 'categories', 'categories_tags', 'categories_en',
       'countries', 'countries_tags', 'countries_en', 'ingredients_text',
       'ingredients_tags', 'ingredients_analysis_tags', 'additives_n',
       'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'pnns_groups_2',
       'states', 'states_tags', 'states_en', 'ecoscore_grade', 'completeness',
       'last_image_t', 'last_image_datetime', 'main_category',
       'main_category_en', 'image_url', 'image_small_url',
       'image_ingredients_url', 'image_ingredients_small_url', 'energy_100g',
       'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g',
       'fruits-vegetables-nuts-estimate-from-ingredients_100g'],
      dtype='object'

In [10]:
columns_to_keep = [
    "code",
    "url",
    "last_modified_datetime",
    "product_name",
    "packaging_tags",
    "categories",
    "categories_tags",
    "ingredients_tags",
    "ingredients_analysis_tags",
    "food_groups_tags",
    "main_category",
    "allergens",
    "additives_tags",
    "traces_tags",
    "nutriscore_grade",
    "ecoscore_grade",
    "nutrient_levels_tags",
    "image_url",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
    "fruits-vegetables-nuts-estimate-from-ingredients_100g",
]
data_en_filtered = data_en_filtered[columns_to_keep]

In [11]:
print(len(data_en_filtered))
# Remove deplicates lines from the dataset
data_en_filtered = data_en_filtered.drop_duplicates()
print(len(data_en_filtered))

255537
255059


In [12]:
print(len(data_en_filtered))
# Remove the duplicates code in the dataset
data_en_filtered = data_en_filtered.drop_duplicates(subset="code")
print(len(data_en_filtered))

255059
254765


In [13]:
# Save the data_en_filtered
data_en_filtered.to_csv("../data/filtered_dataset_openfoodfacts_en.csv", index=False)

### Data Cleaning

In [13]:
data_en_filtered = pd.read_csv("../data/filtered_dataset_openfoodfacts_en.csv", dtype={'code': str})

In [14]:
data_en_filtered.head(2)

,code,url,last_modified_datetime,product_name,packaging_tags,categories,categories_tags,ingredients_tags,ingredients_analysis_tags,food_groups_tags,main_category,allergens,additives_tags,traces_tags,nutriscore_grade,ecoscore_grade,nutrient_levels_tags,image_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
0,5,http://world-en.openfoodfacts.org/product/00000005/bio-inulin-ewl,2024-10-20T14:02:34Z,Bio inulin,en:glass,Gemüse,"en:plant-based-foods-and-beverages,en:plant-based-foods,en:fruits-and-vegetables-based-foods,en:vegetables-based-foods,en:vegetables","en:apricot-juice-and-puree,en:fruit,en:prunus-species-fruit,en:apricot,en:apricot-juice,en:apricot-puree,en:water,en:sugar,en:added-sugar,en:disaccharide","en:palm-oil-free,en:vegan,en:vegetarian","en:fruits-and-vegetables,en:vegetables",en:vegetables,NaN,NaN,NaN,a,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low-quantity,en:sugars-in-moderate-quantity,en:salt-in-low-quantity",https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.400.jpg,840.0,0.0,0.0,8.0,8.0,0.0,0.23,50.000000
1,10,http://world-en.openfoodfacts.org/product/00000010/madeleines-nature-bijou,2024-10-04T09:38:47Z,Madeleines nature,"en:plastic,en:cardboard,fr:boite-en-carton,fr:film-en-plastique","Snacks, Desserts, Snacks sucrés, Biscuits et gâteaux, Gâteaux, Madeleines, Madeleines natures","en:snacks,en:desserts,en:sweet-snacks,en:biscuits-and-cakes,en:cakes,en:madeleines,en:plain-madeleines","en:wheat-flour,en:cereal,en:flour,en:wheat,en:cereal-flour,en:sugar,en:added-sugar,en:disaccharide,en:colza-oil,en:oil-and-fat,en:vegetable-oil-and-fat,en:rapeseed-oil,en:free-range-chicken-eggs,en:egg,en:chicken-egg,en:free-range-eggs,en:glucose-fructose-syrup,en:monosaccharide,en:fructose,en:glucose,en:stabiliser,en:raising-agent,en:e331,en:salt,en:wheat-gluten,en:gluten,en:skimmed-milk-powder,en:dairy,en:milk-powder,en:vegetable-fiber,en:fiber,en:natural-flavouring,en:flavouring,en:e422,fr:carbonates-d-ammonium-carbonates-de-sodium","en:palm-oil-free,en:non-vegan,en:vegetarian-status-unknown","en:sugary-snacks,en:biscuits-and-cakes",en:plain-madeleines,"en:eggs,en:gluten,en:milk","en:e331,en:e422,en:e503","en:nuts,en:soybeans",d,c,"en:fat-in-high-quantity,en:saturated-fat-in-moderate-quantity,en:sugars-in-high-quantity,en:salt-in-moderate-quantity",https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.400.jpg,1852.0,22.0,2.6,54.0,25.0,6.4,0.53,22.666667


In [6]:
def catetegorial_and_numerical_features(df):
    categorical_features = df.select_dtypes(include=['object', 'bool']).columns
    numerical_features = df.select_dtypes(include=['number']).columns
    return categorical_features, numerical_features

In [17]:
categorical_feature, numerical_feature = catetegorial_and_numerical_features(data_en_filtered)

In [18]:
categorical_feature

Index(['code', 'url', 'last_modified_datetime', 'product_name',
       'packaging_tags', 'categories', 'categories_tags', 'ingredients_tags',
       'ingredients_analysis_tags', 'food_groups_tags', 'main_category',
       'allergens', 'additives_tags', 'traces_tags', 'nutriscore_grade',
       'ecoscore_grade', 'nutrient_levels_tags', 'image_url'],
      dtype='object')

In [15]:
def clean_dataset(df):
    # Convertir les colonnes de date en années et gérer les erreurs
    df['last_modified_year'] = pd.to_datetime(df['last_modified_datetime'], errors='coerce').dt.year
    df.drop(['last_modified_datetime'], axis=1, inplace=True)

    # Liste des colonnes à remplir avec une valeur par défaut
    fill_unknown = [
        "packaging_tags",
        "categories_tags",
        "ingredients_tags",
        "ingredients_analysis_tags",
        "nutriscore_grade",
        "ecoscore_grade",
        "nutrient_levels_tags",
        "categories",
    ]
    fill_none = [
        "allergens",
        "traces_tags",
        "additives_tags",
        "food_groups_tags",
        "main_category",
        "image_url",
    ]

    # Remplacer les valeurs manquantes
    df[fill_unknown] = df[fill_unknown].fillna("unknown")
    df[fill_none] = df[fill_none].fillna("none")

    return df


# Appliquer la fonction
data_en_filtered_clean = clean_dataset(data_en_filtered)

In [20]:
to_split = [
    "packaging_tags",
    "categories_tags",
    "ingredients_tags",
    "ingredients_analysis_tags",
    "allergens",
    "additives_tags",
    "nutrient_levels_tags",
    "traces_tags",
    "main_category",
    "food_groups_tags",
    "categories",
]

In [21]:
for feature in to_split:
    data_en_filtered_clean[feature] = data_en_filtered_clean[feature].fillna("").str.replace("en:", "").str.split(",")

In [22]:
def select_frequent_tags(data, column_name, min_count=100, min_for_various_tags=100):
    all_tags = [item for items in data[column_name].dropna() for item in items]
    tag_counts = Counter(all_tags)
    frequent_tags = [tag for tag, count in tag_counts.items() if count >= min_count]
    if len(frequent_tags) > min_for_various_tags:
        frequent_tags = [tag for tag, _ in tag_counts.most_common(min_for_various_tags)]
    frequent_tags.append("other")
    return frequent_tags

In [23]:
# Applique cette fonction aux colonnes de to_split
frequent_tags = {
    col: select_frequent_tags(data_en_filtered_clean, col) for col in to_split
}

In [24]:
data_en_filtered_clean["food_groups_tags"].value_counts().reset_index().head()

,food_groups_tags,count
0,[none],56514
1,"[sugary-snacks, biscuits-and-cakes]",17164
2,"[sugary-snacks, sweets]",16469
3,"[composite-foods, one-dish-meals]",15040
4,"[fish-meat-eggs, processed-meat]",10615


In [25]:
len(frequent_tags["food_groups_tags"])

51

#### First dataset using boolean labels

In [26]:
#  Transformation en colonnes booléennes seulement pour les tags fréquents
new_columns = {}

for col, tags in frequent_tags.items():
    for tag in tags:
        new_columns[f"{col}_{tag}"] = data_en_filtered_clean[col].apply(
            lambda x: tag in x if isinstance(x, list) else False
        )

    # Créer une colonne booléenne pour les tags non fréquents
    new_columns[f"{col}_other"] = data_en_filtered[col].apply(
        lambda x: any(item not in tags for item in x) if isinstance(x, list) else False
    )

# Convert the dictionary to a DataFrame
new_columns_df = pd.DataFrame(new_columns)

In [27]:
print(data_en_filtered.shape)

print(new_columns_df.shape)

(254765, 26)
(254765, 723)


In [28]:
frequent_tags_list = list(frequent_tags.keys())

In [29]:
frequent_tags_list

['packaging_tags',
 'categories_tags',
 'ingredients_tags',
 'ingredients_analysis_tags',
 'allergens',
 'additives_tags',
 'nutrient_levels_tags',
 'traces_tags',
 'main_category',
 'food_groups_tags',
 'categories']

In [30]:
data_en_filtered_clean.head(2)

,code,url,product_name,packaging_tags,categories,categories_tags,ingredients_tags,ingredients_analysis_tags,food_groups_tags,main_category,allergens,additives_tags,traces_tags,nutriscore_grade,ecoscore_grade,nutrient_levels_tags,image_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year
0,5,http://world-en.openfoodfacts.org/product/00000005/bio-inulin-ewl,Bio inulin,[glass],[Gemüse],"[plant-based-foods-and-beverages, plant-based-foods, fruits-and-vegetables-based-foods, vegetables-based-foods, vegetables]","[apricot-juice-and-puree, fruit, prunus-species-fruit, apricot, apricot-juice, apricot-puree, water, sugar, added-sugar, disaccharide]","[palm-oil-free, vegan, vegetarian]","[fruits-and-vegetables, vegetables]",[vegetables],[none],[none],[none],a,unknown,"[fat-in-low-quantity, saturated-fat-in-low-quantity, sugars-in-moderate-quantity, salt-in-low-quantity]",https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.400.jpg,840.0,0.0,0.0,8.0,8.0,0.0,0.23,50.000000,2024
1,10,http://world-en.openfoodfacts.org/product/00000010/madeleines-nature-bijou,Madeleines nature,"[plastic, cardboard, fr:boite-en-carton, fr:film-en-plastique]","[Snacks, Desserts, Snacks sucrés, Biscuits et gâteaux, Gâteaux, Madeleines, Madeleines natures]","[snacks, desserts, sweet-snacks, biscuits-and-cakes, cakes, madeleines, plain-madeleines]","[wheat-flour, cereal, flour, wheat, cereal-flour, sugar, added-sugar, disaccharide, colza-oil, oil-and-fat, vegetable-oil-and-fat, rapeseed-oil, free-range-chicken-eggs, egg, chicken-egg, free-range-eggs, glucose-fructose-syrup, monosaccharide, fructose, glucose, stabiliser, raising-agent, e331, salt, wheat-gluten, gluten, skimmed-milk-powder, dairy, milk-powder, vegetable-fiber, fiber, natural-flavouring, flavouring, e422, fr:carbonates-d-ammonium-carbonates-de-sodium]","[palm-oil-free, non-vegan, vegetarian-status-unknown]","[sugary-snacks, biscuits-and-cakes]",[plain-madeleines],"[eggs, gluten, milk]","[e331, e422, e503]","[nuts, soybeans]",d,c,"[fat-in-high-quantity, saturated-fat-in-moderate-quantity, sugars-in-high-quantity, salt-in-moderate-quantity]",https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.400.jpg,1852.0,22.0,2.6,54.0,25.0,6.4,0.53,22.666667,2024


In [31]:
categorical_feature, numerical_feature = catetegorial_and_numerical_features(data_en_filtered_clean)

In [32]:
categorical_feature

Index(['code', 'url', 'product_name', 'packaging_tags', 'categories',
       'categories_tags', 'ingredients_tags', 'ingredients_analysis_tags',
       'food_groups_tags', 'main_category', 'allergens', 'additives_tags',
       'traces_tags', 'nutriscore_grade', 'ecoscore_grade',
       'nutrient_levels_tags', 'image_url'],
      dtype='object')

In [33]:
# Drop columns listed in `frequent_tags_list` and display the first few rows
dataset_with_bool = data_en_filtered_clean.drop(columns=frequent_tags_list)

In [34]:
dataset_with_bool.head()

,code,url,product_name,nutriscore_grade,ecoscore_grade,image_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year
0,5,http://world-en.openfoodfacts.org/product/00000005/bio-inulin-ewl,Bio inulin,a,unknown,https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.400.jpg,840.0,0.0,0.0,8.0,8.0,0.0,0.23,50.000000,2024
1,10,http://world-en.openfoodfacts.org/product/00000010/madeleines-nature-bijou,Madeleines nature,d,c,https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.400.jpg,1852.0,22.0,2.6,54.0,25.0,6.4,0.53,22.666667,2024
2,15,http://world-en.openfoodfacts.org/product/00000015/madeleines-chocolait-bijou,Madeleines ChocoLait,d,d,https://images.openfoodfacts.org/images/products/000/000/000/0015/front_fr.22.400.jpg,1926.0,24.0,6.0,54.0,31.0,6.4,0.48,16.250000,2024
3,20,http://world-en.openfoodfacts.org/product/00000020/madeleines-choco-noir-bijou,Madeleines Choco Noir,d,d,https://images.openfoodfacts.org/images/products/000/000/000/0020/front_fr.39.400.jpg,1953.0,25.0,6.1,53.0,29.0,6.3,0.45,16.250000,2024
4,22,http://world-en.openfoodfacts.org/product/00000022/farandole-de-madeleine-bijou,Farandole de madeleine,unknown,d,https://images.openfoodfacts.org/images/products/000/000/000/0022/front_fr.3.400.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.750000,2024


In [35]:
dataset_with_bool = pd.concat([dataset_with_bool, new_columns_df], axis=1)

In [36]:
# Save the dataset cleaned
dataset_with_bool.to_csv("../data/dataset_openfoodfacts_bool_cleaned.csv", index=False)

#### Second dataset using NLP TFIDF

In [37]:
data_en_filtered_clean.head(2)

,code,url,product_name,packaging_tags,categories,categories_tags,ingredients_tags,ingredients_analysis_tags,food_groups_tags,main_category,allergens,additives_tags,traces_tags,nutriscore_grade,ecoscore_grade,nutrient_levels_tags,image_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year
0,5,http://world-en.openfoodfacts.org/product/00000005/bio-inulin-ewl,Bio inulin,[glass],[Gemüse],"[plant-based-foods-and-beverages, plant-based-foods, fruits-and-vegetables-based-foods, vegetables-based-foods, vegetables]","[apricot-juice-and-puree, fruit, prunus-species-fruit, apricot, apricot-juice, apricot-puree, water, sugar, added-sugar, disaccharide]","[palm-oil-free, vegan, vegetarian]","[fruits-and-vegetables, vegetables]",[vegetables],[none],[none],[none],a,unknown,"[fat-in-low-quantity, saturated-fat-in-low-quantity, sugars-in-moderate-quantity, salt-in-low-quantity]",https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.400.jpg,840.0,0.0,0.0,8.0,8.0,0.0,0.23,50.000000,2024
1,10,http://world-en.openfoodfacts.org/product/00000010/madeleines-nature-bijou,Madeleines nature,"[plastic, cardboard, fr:boite-en-carton, fr:film-en-plastique]","[Snacks, Desserts, Snacks sucrés, Biscuits et gâteaux, Gâteaux, Madeleines, Madeleines natures]","[snacks, desserts, sweet-snacks, biscuits-and-cakes, cakes, madeleines, plain-madeleines]","[wheat-flour, cereal, flour, wheat, cereal-flour, sugar, added-sugar, disaccharide, colza-oil, oil-and-fat, vegetable-oil-and-fat, rapeseed-oil, free-range-chicken-eggs, egg, chicken-egg, free-range-eggs, glucose-fructose-syrup, monosaccharide, fructose, glucose, stabiliser, raising-agent, e331, salt, wheat-gluten, gluten, skimmed-milk-powder, dairy, milk-powder, vegetable-fiber, fiber, natural-flavouring, flavouring, e422, fr:carbonates-d-ammonium-carbonates-de-sodium]","[palm-oil-free, non-vegan, vegetarian-status-unknown]","[sugary-snacks, biscuits-and-cakes]",[plain-madeleines],"[eggs, gluten, milk]","[e331, e422, e503]","[nuts, soybeans]",d,c,"[fat-in-high-quantity, saturated-fat-in-moderate-quantity, sugars-in-high-quantity, salt-in-moderate-quantity]",https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.400.jpg,1852.0,22.0,2.6,54.0,25.0,6.4,0.53,22.666667,2024


In [38]:
categorical_feature, numerical_feature = catetegorial_and_numerical_features(data_en_filtered_clean)

In [39]:
# Get the from the categorical features the feature which have list as object
text_features = [
    col for col in categorical_feature if isinstance(data_en_filtered_clean[col].iloc[0], list)
]
text_features

['packaging_tags',
 'categories',
 'categories_tags',
 'ingredients_tags',
 'ingredients_analysis_tags',
 'food_groups_tags',
 'main_category',
 'allergens',
 'additives_tags',
 'traces_tags',
 'nutrient_levels_tags']

In [40]:
data_en_filtered_clean.head(2)

,code,url,product_name,packaging_tags,categories,categories_tags,ingredients_tags,ingredients_analysis_tags,food_groups_tags,main_category,allergens,additives_tags,traces_tags,nutriscore_grade,ecoscore_grade,nutrient_levels_tags,image_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year
0,5,http://world-en.openfoodfacts.org/product/00000005/bio-inulin-ewl,Bio inulin,[glass],[Gemüse],"[plant-based-foods-and-beverages, plant-based-foods, fruits-and-vegetables-based-foods, vegetables-based-foods, vegetables]","[apricot-juice-and-puree, fruit, prunus-species-fruit, apricot, apricot-juice, apricot-puree, water, sugar, added-sugar, disaccharide]","[palm-oil-free, vegan, vegetarian]","[fruits-and-vegetables, vegetables]",[vegetables],[none],[none],[none],a,unknown,"[fat-in-low-quantity, saturated-fat-in-low-quantity, sugars-in-moderate-quantity, salt-in-low-quantity]",https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.400.jpg,840.0,0.0,0.0,8.0,8.0,0.0,0.23,50.000000,2024
1,10,http://world-en.openfoodfacts.org/product/00000010/madeleines-nature-bijou,Madeleines nature,"[plastic, cardboard, fr:boite-en-carton, fr:film-en-plastique]","[Snacks, Desserts, Snacks sucrés, Biscuits et gâteaux, Gâteaux, Madeleines, Madeleines natures]","[snacks, desserts, sweet-snacks, biscuits-and-cakes, cakes, madeleines, plain-madeleines]","[wheat-flour, cereal, flour, wheat, cereal-flour, sugar, added-sugar, disaccharide, colza-oil, oil-and-fat, vegetable-oil-and-fat, rapeseed-oil, free-range-chicken-eggs, egg, chicken-egg, free-range-eggs, glucose-fructose-syrup, monosaccharide, fructose, glucose, stabiliser, raising-agent, e331, salt, wheat-gluten, gluten, skimmed-milk-powder, dairy, milk-powder, vegetable-fiber, fiber, natural-flavouring, flavouring, e422, fr:carbonates-d-ammonium-carbonates-de-sodium]","[palm-oil-free, non-vegan, vegetarian-status-unknown]","[sugary-snacks, biscuits-and-cakes]",[plain-madeleines],"[eggs, gluten, milk]","[e331, e422, e503]","[nuts, soybeans]",d,c,"[fat-in-high-quantity, saturated-fat-in-moderate-quantity, sugars-in-high-quantity, salt-in-moderate-quantity]",https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.400.jpg,1852.0,22.0,2.6,54.0,25.0,6.4,0.53,22.666667,2024


In [41]:
# Définir les paramètres pour la taille maximale du vocabulaire
max_vocab = 100
min_vocab = 50

# Initialiser un DataFrame vide pour stocker les résultats TF-IDF
tfidf_results = pd.DataFrame(index=data_en_filtered_clean.index)

# Boucle sur chaque colonne de `text_features` (qui contient des listes de tags)
for col in text_features:
    print(col)
    # Convertir les listes en chaînes de caractères séparées par des espaces pour compatibilité avec TF-IDF
    data_en_filtered_clean[col] = data_en_filtered_clean[col].apply(
        lambda x: " ".join(x) if isinstance(x, list) else ""
    )

    # Déterminer la taille du vocabulaire pour la colonne actuelle
    vectorizer = TfidfVectorizer()
    vectorizer.fit(data_en_filtered_clean[col])
    vocab_size = len(vectorizer.vocabulary_)

    # Définir `max_features` selon la taille du vocabulaire
    max_features = min(max_vocab, vocab_size) if vocab_size >= min_vocab else vocab_size

    # Reconfigurer TfidfVectorizer avec le nombre maximal de termes si nécessaire
    vectorizer = TfidfVectorizer(max_features=max_features)
    tfidf_matrix = vectorizer.fit_transform(data_en_filtered_clean[col])

    # Convertir la matrice TF-IDF en DataFrame avec noms de colonnes appropriés
    tfidf_df = pd.DataFrame(
        tfidf_matrix.toarray(),
        columns=[f"{col}_{term}" for term in vectorizer.get_feature_names_out()],
        index=data_en_filtered_clean.index,
    )

    # Ajouter le DataFrame TF-IDF aux résultats
    tfidf_results = pd.concat([tfidf_results, tfidf_df], axis=1)
    
    print('Done')

packaging_tags


Done
categories
Done
categories_tags
Done
ingredients_tags
Done
ingredients_analysis_tags
Done
food_groups_tags
Done
main_category
Done
allergens
Done
additives_tags
Done
traces_tags
Done
nutrient_levels_tags
Done


In [42]:
dataset_with_tfidf = data_en_filtered_clean.drop(columns=text_features)

In [43]:
dataset_with_tfidf.head()

,code,url,product_name,nutriscore_grade,ecoscore_grade,image_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year
0,5,http://world-en.openfoodfacts.org/product/00000005/bio-inulin-ewl,Bio inulin,a,unknown,https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.400.jpg,840.0,0.0,0.0,8.0,8.0,0.0,0.23,50.000000,2024
1,10,http://world-en.openfoodfacts.org/product/00000010/madeleines-nature-bijou,Madeleines nature,d,c,https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.400.jpg,1852.0,22.0,2.6,54.0,25.0,6.4,0.53,22.666667,2024
2,15,http://world-en.openfoodfacts.org/product/00000015/madeleines-chocolait-bijou,Madeleines ChocoLait,d,d,https://images.openfoodfacts.org/images/products/000/000/000/0015/front_fr.22.400.jpg,1926.0,24.0,6.0,54.0,31.0,6.4,0.48,16.250000,2024
3,20,http://world-en.openfoodfacts.org/product/00000020/madeleines-choco-noir-bijou,Madeleines Choco Noir,d,d,https://images.openfoodfacts.org/images/products/000/000/000/0020/front_fr.39.400.jpg,1953.0,25.0,6.1,53.0,29.0,6.3,0.45,16.250000,2024
4,22,http://world-en.openfoodfacts.org/product/00000022/farandole-de-madeleine-bijou,Farandole de madeleine,unknown,d,https://images.openfoodfacts.org/images/products/000/000/000/0022/front_fr.3.400.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.750000,2024


In [44]:
dataset_with_tfidf = pd.concat([dataset_with_tfidf, tfidf_results], axis=1)

In [45]:
# Save the dataset
dataset_with_tfidf.to_csv("../data/dataset_openfoodfacts_tfidf_cleaned.csv", index=False)

In [46]:
dataset_with_tfidf.head()

,code,url,product_name,nutriscore_grade,ecoscore_grade,image_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year,packaging_tags_aluminium,packaging_tags_au,packaging_tags_bac,packaging_tags_bag,packaging_tags_barquette,packaging_tags_blister,packaging_tags_bocal,packaging_tags_boite,packaging_tags_bottle,packaging_tags_bouchon,packaging_tags_bouteille,packaging_tags_box,packaging_tags_brick,packaging_tags_brique,packaging_tags_can,packaging_tags_canned,packaging_tags_cap,packaging_tags_capsule,packaging_tags_card,packaging_tags_cardboard,packaging_tags_carton,packaging_tags_container,packaging_tags_conteneur,packaging_tags_cork,packaging_tags_couvercle,packaging_tags_de,packaging_tags_dot,packaging_tags_drink,packaging_tags_dry,packaging_tags_emballage,packaging_tags_en,packaging_tags_envelope,packaging_tags_et,packaging_tags_etui,packaging_tags_facile,packaging_tags_feuille,packaging_tags_film,packaging_tags_flacon,packaging_tags_flask,packaging_tags_food,packaging_tags_fr,packaging_tags_fresh,packaging_tags_frozen,packaging_tags_gas,packaging_tags_glass,packaging_tags_green,packaging_tags_individual,packaging_tags_jar,packaging_tags_jeter,packaging_tags_lid,packaging_tags_metal,packaging_tags_metals,packaging_tags_mixed,packaging_tags_opercule,packaging_tags_or,packaging_tags_ouverture,packaging_tags_pack,packaging_tags_packed,packaging_tags_packet,packaging_tags_pak,packaging_tags_paper,packaging_tags_papier,packaging_tags_pensez,packaging_tags_pet,packaging_tags_plastic,packaging_tags_plastique,packaging_tags_point,packaging_tags_polyethylene,packaging_tags_polypropylene,packaging_tags_pot,packaging_tags_pots,packaging_tags_pouch,packaging_tags_pp,packaging_tags_protective,packaging_tags_recyclable,packaging_tags_recycle,packaging_tags_recycler,packaging_tags_refrigerated,packaging_tags_sac,packaging_tags_sachet,packaging_tags_sachets,packaging_tags_seal,packaging_tags_sheet,packaging_tags_sleeve,packaging_tags_sous,packaging_tags_steel,packaging_tags_terephthalate,packaging_tags_tetra,packaging_tags_tray,packaging_tags_tri,packaging_tags_triman,packaging_tags_tube,packaging_tags_unknown,packaging_tags_vacuum,packaging_tags_verre,packaging_tags_vert,packaging_tags_vial,packaging_tags_wine,packaging_tags_wood,packaging_tags_wrapper,categories_ajouté,categories_alimentaires,categories_aliments,categories_and,categories_au,categories_aux,categories_avec,categories_base,categories_based,categories_beverages,categories_biscuits,categories_bières,categories_blancs,categories_boissons,categories_bonbons,categories_bœuf,categories_cacao,categories_charcuteries,categories_chips,categories_chocolat,categories_chocolats,categories_compotes,categories_condiments,categories_confiseries,categories_confitures,categories_conserve,categories_conserves,categories_coques,categories_crèmes,categories_céréales,categories_de,categories_desserts,categories_déjeuners,categories_dérivés,categories_déshydratés,categories_en,categories_et,categories_fermentés,categories_foies,categories_foods,categories_frais,categories_fromages,categories_fruits,categories_graines,categories_gras,categories_grasses,categories_groceries,categories_gâteaux,categories_huiles,categories_jambons,categories_jus,categories_la,categories_lactés,categories_lait,categories_laitiers,categories_laits,categories_légumes,categories_légumineuses,categories_matières,categories_mer,categories_nectars,categories_origine,categories_pains,categories_petit,categories_pizzas,categories_plant,categories_plantes,categories_plats,categories_poissons,categories_pommes,categories_porc,categories_poulet,categories_pour,categories_products,categories_produits,categories_préparations,categories_préparés,categories_pâtes,categories_riz,categories_salés,categories_sans,categories_sauces,categories_secs,categories_snacks,categories_soupes,categories_sucre,categories_sucrés,catego

#### Preprocess the dataset_openfoodfacts_bool_cleaned and apply kmeans

In [2]:
dataset_with_bool = pd.read_csv("../data/dataset_openfoodfacts_bool_cleaned.csv", dtype={'code': str})

In [3]:
dataset_with_bool.head(1)

,code,url,product_name,nutriscore_grade,ecoscore_grade,image_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year,packaging_tags_unknown,packaging_tags_plastic,packaging_tags_cardboard,packaging_tags_bag,packaging_tags_fresh,packaging_tags_glass,packaging_tags_tray,packaging_tags_box,packaging_tags_bottle,packaging_tags_metal,packaging_tags_frozen,packaging_tags_pot,packaging_tags_jar,packaging_tags_canned,packaging_tags_fr:film-en-plastique,packaging_tags_protective-gas,packaging_tags_fr:sachet-plastique,packaging_tags_fr:etui-en-carton,packaging_tags_film,packaging_tags_paper,packaging_tags_brick,packaging_tags_sleeve,packaging_tags_vacuum-packed,packaging_tags_lid,packaging_tags_recyclable-metals,packaging_tags_aluminium,packaging_tags_fr:film-plastique,packaging_tags_seal,packaging_tags_fr:sachet-en-plastique,packaging_tags_bottle-cap,packaging_tags_wine-cork,packaging_tags_fr:barquette-plastique,packaging_tags_fr:bocal-en-verre,packaging_tags_vial,packaging_tags_can,packaging_tags_fr:pot-en-verre,packaging_tags_fr:0,packaging_tags_fr:point-vert,packaging_tags_fr:bouteille-en-verre,packaging_tags_pack,packaging_tags_dry,packaging_tags_fr:pensez-au-tri,packaging_tags_fr:boite-en-carton,packaging_tags_0,packaging_tags_fr:boite-carton,packaging_tags_fr:conteneur,packaging_tags_fr:pot-en-plastique,packaging_tags_pouch-flask,packaging_tags_fr:couvercle-en-metal,packaging_tags_fr:etui-carton-a-recycler,packaging_tags_drink-can,packaging_tags_fr:etui-carton,packaging_tags_fr:bocal-verre,packaging_tags_fr:pot-plastique,packaging_tags_fr:bouteille-plastique,packaging_tags_fr:bouteille-en-plastique,packaging_tags_fr:bouteille-verre,packaging_tags_capsule,packaging_tags_tetra-pak,packaging_tags_container,packaging_tags_envelope,packaging_tags_fr:couvercle-metal,packaging_tags_fr:opercule-en-plastique,packaging_tags_fr:opercule-en-metal,packaging_tags_fr:capsule-en-metal,packaging_tags_wood,packaging_tags_fr:barquette-en-plastique,packaging_tags_fr:triman,packaging_tags_fr:brique-en-carton,packaging_tags_fr:sachet-plastique-a-jeter,packaging_tags_fr:bouchon-en-plastique,packaging_tags_fr:film-plastique-a-jeter,packaging_tags_green-dot,packaging_tags_individual-bag,packaging_tags_recycle,packaging_tags_bottle-or-vial,packaging_tags_refrigerated,packaging_tags_wrapper,packaging_tags_fr:boite-plastique,packaging_tags_fr:boite-en-metal,packaging_tags_sheet,packaging_tags_blister,packaging_tags_fr:barquette-et-film-plastique-a-jeter,packaging_tags_food-can,packaging_tags_unfrozen,packaging_tags_discard,packaging_tags_fr:pot-verre,packaging_tags_fr:bouchon-plastique,packaging_tags_pet-polyethylene-terephthalate,packaging_tags_steel,packaging_tags_net,packaging_tags_fr:boite-metal,packaging_tags_fr:couvercle-plastique,packaging_tags_tube,packaging_tags_fr:sachet-papier,packaging_tags_plastique,packaging_tags_fr:couvercle-ouverture-facile,packaging_tags_card-box,packaging_tags_jug,packaging_tags_fr:flacon-verre,packaging_tags_other,categories_tags_plant-based-foods-and-beverages,categories_tags_plant-based-foods,categories_tags_snacks,categories_tags_unknown,categories_tags_sweet-snacks,categories_tags_meats-and-their-products,categories_tags_beverages,categories_tags_meals,categories_tags_dairies,categories_tags_fruits-and-vegetables-based-foods,categories_tags_meats,categories_tags_cereals-and-potatoes,categories_tags_desserts,categories_tags_fermented-foods,categories_tags_biscuits-and-cakes,categories_tags_fermented-milk-products,categories_tags_frozen-foods,categories_tags_spreads,categories_tags_prepared-meats,categories_tags_cereals-and-their-products,categories_tags_condiments,categories_tags_breakfasts,categories_tags_fruits-based-foods,categories_tags_cheeses,categories_tags_plant-based-beverages,categories_tags_groceries,categories_tags_seafood,categories_tags_confectioneries,categories_tags_vegetables-based-foods,

In [4]:
dataset_with_bool_ml = dataset_with_bool.drop(columns=["code", "url", "image_url", "product_name"])

In [7]:
# Separate feature types
from sklearn.impute import SimpleImputer


categorical_features, numeric_features = catetegorial_and_numerical_features(dataset_with_bool_ml)

# 1. Preprocessor for Numeric and Categorical Features
preprocessor = ColumnTransformer(
    transformers=[
        # Pipeline for numeric features: impute missing values and scale
        (
            "num",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="mean")),
                    ("scaler", StandardScaler()),
                ]
            ),
            numeric_features,
        ),
        # Pipeline for categorical features: impute missing values and one-hot encode
        (
            "cat",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
                ]
            ),
            categorical_features,
        ),
    ]
)
# 2. Full Pipeline including Preprocessing and KMeans
kmeans_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("pca", PCA(n_components=50)),  # Optional: Reduce dimensionality before clustering
        ("kmeans", KMeans(n_clusters=5, random_state=42)),  # Set the number of clusters
    ]
)

# 3. Fit the pipeline on the data
kmeans_pipeline.fit(dataset_with_bool_ml)

# 4. Get cluster labels
cluster_labels = kmeans_pipeline.predict(dataset_with_bool_ml)

# Append cluster labels to the original DataFrame for analysis
dataset_with_bool_ml["cluster"] = cluster_labels

#### Preprocess the dataset_openfoodfacts_tfidf_cleaned and apply kmeans

In [8]:
dataset_with_tfidf = pd.read_csv("../data/dataset_openfoodfacts_tfidf_cleaned.csv", dtype={'code': str})

In [9]:
dataset_with_tfidf_ml = dataset_with_tfidf.drop(columns=["code", "url", "image_url", "product_name"])

In [10]:
# Separate feature types
categorical_features, numeric_features = catetegorial_and_numerical_features(
    dataset_with_tfidf_ml
)

# 1. Preprocessor for Numeric and Categorical Features
preprocessor = ColumnTransformer(
    transformers=[
        # Pipeline for numeric features: impute missing values and scale
        (
            "num",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="mean")),
                    ("scaler", StandardScaler()),
                ]
            ),
            numeric_features,
        ),
        # Pipeline for categorical features: impute missing values and one-hot encode
        (
            "cat",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
                ]
            ),
            categorical_features,
        ),
    ]
)

# 2. Full Pipeline including Preprocessing and KMeans
kmeans_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("pca", PCA(n_components=50)),
        ("kmeans", KMeans(n_clusters=5, random_state=42)),  # Set the number of clusters
    ]
)

# 3. Fit the pipeline on the data
kmeans_pipeline.fit(dataset_with_tfidf_ml)

# 4. Get cluster labels
cluster_labels = kmeans_pipeline.predict(dataset_with_tfidf_ml)

# Append cluster labels to the original DataFrame for analysis
dataset_with_tfidf_ml["cluster"] = cluster_labels

In [11]:
dataset_with_tfidf_ml["cluster"].value_counts()

dataset_with_tfidf["cluster"] = dataset_with_tfidf_ml["cluster"]

dataset_with_tfidf[["product_name"]][dataset_with_tfidf["cluster"] == 1].head(20)

,product_name
0,Bio inulin
7,Organic pea protein powder
11,Fondants Citron
19,Confiture de fraise mara des bois
21,Pan
29,Tisane nerf - sommeil
31,huile végétale
41,pot au feu de légumes aux aromates
42,velouté de 10 légumes
43,Velouté de légumes façon poêlée


In [19]:
datatemp = data_en_filtered_clean["categories"].value_counts().reset_index()
datatemp.head(20)

,categories,count
0,unknown,34935
1,Boissons,1607
2,Surgelés,1304
3,"Snacks, Snacks sucrés, Confiseries, Bonbons",1055
4,"Snacks, Snacks sucrés, Cacao et dérivés, Chocolats, Chocolats noirs",1048
5,"Snacks, Snacks sucrés, Biscuits et gâteaux, Biscuits",987
6,"Viandes et dérivés, Viandes, Charcuteries, Jambons, Jambons blancs",941
7,"Boissons, Boissons avec sucre ajouté",873
8,"Snacks, Snacks salés, Amuse-gueules, Biscuits apéritifs",682
9,"Snacks, Snacks sucrés, Cacao et dérivés, Chocolats, Chocolats au lait",510
